In [1]:
from PIL import Image
import sys
import os

## Data

In [2]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("getomni-ai/ocr-benchmark")

print(type(ds))

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'datasets.dataset_dict.DatasetDict'>


In [3]:
data = ds['test']

## Models

### apple-ocr

In [7]:
module_path = './models/apple-ocr/src'
sys.path.append(os.path.abspath(module_path))
from apple_ocr import ocr

def apple_ocr_handler(image):
  apple_ocr = ocr.OCR(image=image)
  df = apple_ocr.recognize()
  # [(text: str, confidence: float, [bbox.origin.x: float, bbox.origin.y: float, bbox.size.width: float, bbox.size.height: float]), ...]
  out = apple_ocr.data
  return out

In [ ]:
image = data[0]['image']
apple_ocr_handler(image)

(<PIL.PngImagePlugin.PngImageFile image mode=RGB size=1009x417>,
 [('Rank',
   1.0,
   [0.02574257425742574,
    0.8851674641148325,
    0.08316831683168317,
    0.052631578947368474]),
  ('1',
   1.0,
   [0.05742574257425743,
    0.7894736842105263,
    0.023762376237623763,
    0.04784688995215314]),
  ('2',
   1.0,
   [0.05742574257425743,
    0.6842105263157895,
    0.025742574257425745,
    0.04784688995215303]),
  ('3',
   1.0,
   [0.055445544554455446,
    0.5837320574162679,
    0.025742574257425745,
    0.04784688995215314]),
  ('4',
   1.0,
   [0.05742574257425743,
    0.47846889952153115,
    0.023762376237623763,
    0.05263157666585094]),
  ('5',
   1.0,
   [0.055445544554455446,
    0.3732057416267942,
    0.025742574257425745,
    0.04784688995215314]),
  ('6',
   1.0,
   [0.05742574257425743,
    0.26794258373205737,
    0.023762376237623763,
    0.052631578947368474]),
  ('7',
   1.0,
   [0.055445544554455446,
    0.11961722488038273,
    0.027722772277227727,
    0.04

### Easy-OCR

In [ ]:
module_path = './models/apple-ocr/src'
sys.path.append(os.path.abspath(module_path))
from apple_ocr import ocr